In [0]:
# define ADLS API Path
source = "abfss://source@dlsdatabrickseteteamea.dfs.core.windows.net"
bronze = "abfss://bronze@dlsdatabrickseteteamea.dfs.core.windows.net"
silver = "abfss://silver@dlsdatabrickseteteamea.dfs.core.windows.net"
gold = "abfss://gold@dlsdatabrickseteteamea.dfs.core.windows.net"

# source_orders = source + "orders"
# source_customers = gold + "customers"
# print(source_customers)

In [0]:
df = spark.read.format("parquet").load(f"{source}/orders" )
print(df.count())
display(df)

### **Data Reading**

In [0]:
# Cell ที่ 3 มีการอ่านไฟล์จาก container source/orders โดยใช้ Spark Structured Streaming และ Auto Loader เพื่อเก็บ Checkpoint ของไฟล์ที่ได้อ่านแล้ว ซึ่งจะช่วยให้ไม่ต้องอ่านไฟล์เดิมซ้ำ
# readStream ใช้สำหรับการอ่านข้อมูลแบบสตรีมมิ่ง ซึ่งช่วยให้สามารถประมวลผลข้อมูลที่เข้ามาใหม่ได้อย่างต่อเนื่อง
# cloudFiles เป็นฟอร์แมตที่ใช้กับ Auto Loader ของ Databricks ซึ่งช่วยในการจัดการไฟล์ที่เข้ามาใหม่โดยอัตโนมัติ และเก็บ Checkpoint เพื่อไม่ให้เกิดการอ่านไฟล์ซ้ำ
df = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "parquet")\
    .option("cloudFiles.schemaLocation", f"{bronze}/checkpoint_orders")\
    .load(f"{source}/orders")


In [0]:
df.writeStream.format("parquet") \
    .outputMode("append") \
    .option("checkpointLocation" , f"{bronze}/checkpoint_orders") \
    .option("path" , f"{bronze}/orders" ) \
    .trigger(once=True) \
    .start()
# .trigger(processingTime="10 seconds")    ให้เขียนทุกๆ 10 วินาที
# .trigger(once=True) เขียนเพียงครั้งเดียวแล้วจบ

In [0]:
df_bronze_orders = spark.read.format("parquet").load(f"{bronze}/orders")
print(df_bronze_orders.count())
display(df_bronze_orders)